# DevCloudの基本的な操作方法
ここではDevCloudの基本的な操作方法を学びます
- 開発ホスト上でのコマンド実行
- DevCloudのエッジコンピューティングノードでのコマンド(ジョブ)実行
- エッジコンピューティングノードを指定してジョブを実行する方法

DevCloud for Edgeは３つのサーバーと１つのストレージで構成されています
1. Development servers
2. Edge computing node servers
3. Job queueing servers
4. 50GB of cloud storage

Development serverは今このJupyter notebookを実行しているマシンです。Xeonサーバーです。  
Edge computing nodeは様々なハードウエア構成のマシンで構成されています。ノードを指定してジョブを送信することでベンチマークなどの評価を行うことが可能です。

<img src=https://devcloud.intel.com/edge/static/images/svg/Edge-howitworks_PN3.svg width=400, height=400></img>

----
## 1. Development workstation上でいくつかコマンドを実行してみる  
Jupyter notebookのセル内で、`!`から始まる行はshellに渡されて実行されます。`pwd`, `ls`など通常のLinuxコマンドを実行できます。  
ここで実行したコマンドはJupyter notebookを実行しているのと同じマシン (=開発ホスト、Xeonマシン)上で実行されます。  

In [ ]:
!pwd
!ls -l
!echo Hello! I like OpenVINO

上のセルで実行したコマンドは、開発ホスト(Xeon server)上で実行されたものです。  

----
## 2. Edge Computing Nodeに簡単なジョブを送って実行させてみる
簡単なジョブスクリプトを作成し、Edge computing node上で実行する手順を学びます。
先ほどと(ほぼ)同じ内容をDevCloudで実行してみましょう。  
まずは実行したい内容のスクリプトファイルを作ります。  
`%%writefile <filename>`で始まるセルは、その中身がそのままファイルに書き出されます (セルの中身と同じ内容のファイルを作ることができます)
ジョブスクリプトの中身は通常のLinux shell scriptと同じです。

In [ ]:
%%writefile job.sh
echo -n 'start dir='
pwd
cd devcloud-workshop-jp
pwd
ls -l
echo Hello! I like OpenVINO


作られたファイルの中身を確認してみましょう

In [ ]:
!cat job.sh

ジョブスクリプトができたので`qsub`コマンドでジョブをキューサーバーに送信します。送信されたキューはEdge compute nodeのうち、いづれかの空いているノードに送られ、処理されます。  
`qsub`コマンドはいくつかのオプションを受け付けますが、ここでは何も指定せずにジョブを送信します。  
実行すると`job ID`が表示されます。`job ID`の最初の数字が`job #`です。

|Option|Description|
|:--|:--|
|`-l` resource_name[=val][,resource_val[=val]]|使用するコンピュートノードを制限(指定)|
|`-o` file|STDOUTを書き出すファイル|
|`-e` file|STDERRを書き出すファイル|
|`-N` job_name|ジョブネーム|
|`-d` working_dir|作業ディレクトリを指定|
|`-F` args_for_job|ジョブスクリプトへの引数|

In [ ]:
!qsub job.sh

実行中のジョブの状態を`qstat`コマンドで確認することができます。ジョブ送信時に`-N`オプションでジョブ名を指定できますが、省略した場合ジョブファイル名がそのままジョブ名になります。  
ジョブが終了するまで(job.shが表示されなくなるまで)定期的にこのセルを実行して、ジョブの終了を待ちます。

In [ ]:
!qstat

ジョブの出力ファイルが作られていることを確認します (`job.sh.o<job#>`)

In [ ]:
!ls -l job.sh.o*

ジョブ出力ファイルの中身を確認します。下記のファイル名の中の、`job#` (数字)の部分は**ジョブを実行するごとに変わります**ので適宜修正する必要があります。`job#`は`qsub`コマンドでジョブを送信したときに表示されています。

例 => `job.sh.o22345`

In [ ]:
!cat job.sh.o<job#>

ここまででDevCloudの一番基礎的な操作方法を学びました。基本的な流れは下記のとおりです。
1. Edge computing nodeで実行したいジョブスクリプトを作る (Shell script)
2. `qsub`コマンドでrequest queue serverにジョブスクリプトを送信
3. `qstat`コマンドで実行状態を確認 (`Q`=`Queued`, `R`=`Running`, 'E'='Error')
4. `<job_name>.o<job#>`ファイルに標準出力の内容がリダイレクトされているので実行結果を確認

**<時間の余った方へ>** `job.sh`をいろいろ書き換えてどんなことができるか試してみよう

### Jobデザイン時のポイント
- Edge computing node側のjob開始時のディレクトリはホームディレクトリ （開発ホストのカレントディレクトリと同じではないので注意)
- Edge computing node側からも開発ホスト側からも同じストレージが見える（ジョブに必要なファイルをいちいち転送したりする必要はない）
- ストレージの内容は使用しているユーザーにしか見えない（インテルにも見えない）。機密情報も扱うことが可能。

----
## 3. エッジコンピュートノードを指定して、特定のノードにジョブを送信する

### 利用可能なコンピュートノードの一覧を表示する
まずは利用可能なコンピュートノード一覧を表示させてみましょう。`pbsnodes`コマンドを使用します
<br><br>
-ノード名
 - 各properties行がノード名のセットになります。
 - たとえば、下記のような行が表示された場合、`uniq`コマンドで重複行をカウントしているので、DevCloud上にこれと同じノードが13台存在しているということがわかります。  
>`13      properties = idc002mx8,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,net1gbe,hddl-r,iei-mustang-v100-mx8`
 - また、このノードを指定するにはカンマで区切られたどの名前でも指定可能です。たとえば`idc002mx8`や`intel-core`、`intel-hd-530`のような名前を指定することが可能です。ノード名を指定してジョブを送信した場合、ジョブはそのノード名を持つ「いづれかの」ノードで実行されます。`intel-core`というノード名はいくつかのほかのノードでも共有されていますので、そのうちの「どれか」で実行されることになります。より狭い名前を指定することで特定のノードを指定することが可能になりますが、ノードがビジーの場合待たされる時間が長くなる可能性があります。


In [ ]:
!pbsnodes | grep "properties =" | sort | uniq -c

### ノードを指定してジョブを送る
- `qsub`コマンド
 - `qsub`コマンドでジョブを送るときに、`-l`(limit)オプションをつけることでジョブを実行するノードを制限(=指定)することができます。`-l nodes=1:node_name`のように指定します。たとえば、`skylake`ノードで実行したい場合、`qsub -l nodes=1:skylake job.sh`のように指定します。
ジョブスクリプトは先に作成した`job.sh`をそのまま利用します。
- ログファイル名
 - ここではPythonを使って`qsub`コマンドから返される`job id` (`['27214.v-qsvr-1.devcloud-edge']`のような文字列のリスト)からログファイル名を自動生成させています
- ジョブの終了検出
 - 前回は手動で`qstat`コマンドを何度も実行し、`job.sh`ジョブがなくなるのを確認しましたが、今回はPythonを使って自動で終了を検出するようにしています

In [ ]:
# submit a job (edge nodeとしてi5-6500teを指定)
job_id=!qsub -l nodes=1:i5-6500te job.sh

# generate log file name from job_id
job_num = job_id[0].split('.')[0]
log_file='job.sh.o'+job_num
err_file='job.sh.e'+job_num
print('job_id={}, log_file={}'.format(job_id, log_file))

import time
def waitForJobCompletion(jobNumber):
    print('Waiting for job completion...', end='')
    running=True
    while running:
        time.sleep(1)
        running=False
        status_list=!qstat         # Check job status
        for status in status_list:
            if jobNumber in status:    # if job_num is found in the status list, the job is still running
                running = True
        print(status.split()[4], end='')
    print('...Job {} completed'.format(job_num))    

# wait for the job to complete
waitForJobCompletion(job_num)

### ジョブ出力ファイルの中身を確認する

表示されたログ情報の中の`Resources:`の行にジョブ実行に使用されたノード名が表示されるので、指定したノードで実行されたかどうかを確認する。

In [ ]:
# 1つ前のセルでlog_fileにログファイル名を入れている。環境変数にセットしてcatから使えるようにする
import os
os.environ['log_file']=log_file

!cat $log_file

----
これでDevCloud for Edgeの基礎的な使い方を習得できました。  
ここでは、下記のことを学びました。
- DevCloudのDevelopment server上でコマンドを実行する方法
- DevCloudのEdge Computing Nodeへのジョブの送信、ジョブステータスの確認、およびジョブ実行結果確認方法

## Next => [OpenVINO BenchmarkをDevCloudで実行する方法](./benchmarking.ipynb)